# Fitting 2D nPRFs 

(n1&n2 number info at timepoint of stim2 presentation/decision)

State of the analysis
- originally small but consequential bug in fit_glm_denoise_stim2.py (slicing of image results)
- fix: rerun GLMsingle - stim2 `glm_stim2.denoise`, & already created `encoding_model.2d.cv.mixture.same_rfs` & `decoding.2d.mixture.same_rfs` [Novermber 24]
- missing: `encoding_model_stim2.cv.denoise` with new betas from GLMsingle


In [1]:
import os.path as op
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

bids_folder = '/shares/zne.uzh/mrenke/ds-dnumrisk'
subjects = [int(f[4:]) for f in os.listdir(bids_folder) if f[0:3] == 'sub' and len(f) == 6]

/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from numrisk.utils.data import get_data_magjduge

df = get_data_magjduge(bids_folder=bids_folder)
df['log(n2)'] = np.log(df['n2'] )

number of subjects found: 66


In [3]:
bids_folder

'/shares/zne.uzh/mrenke/ds-dnumrisk'

In [4]:
n_stim = 2
subject = '01'
key = f'/decoding.2d.mixture.same_rfs'

fn = f'sub-{subject}_mask-NPC_R_n_voxels-100_pdf.tsv'

pdf_fn = op.join(bids_folder, 'derivatives', key, f'sub-{subject}', 'func', fn)
pdf = pd.read_csv(pdf_fn, sep='\t', index_col=[0])

FileNotFoundError: [Errno 2] No such file or directory: '/decoding.2d.mixture.same_rfs/sub-01/func/sub-01_mask-NPC_R_n_voxels-100_pdf.tsv'

In [5]:
pdf_fn = '/shares/zne.uzh/mrenke/ds-dnumrisk/derivatives/decoding.2d.mixture.same_rfs/sub-01/func/sub-01_mask-NPC_R_n_voxels-100_pdf.tsv'
pdf = pd.read_csv(pdf_fn, sep='\t', index_col=[0])

In [6]:
pdf.head()

,Unnamed: 1,1.6094379425048828,1.7917594909667969,1.945910096168518,2.079441547393799,2.1972246170043945,2.3025851249694824,2.397895336151123,2.4849066734313965,2.5649492740631104,...,3.828641414642334.50,3.8501474857330322.50,3.8712010383605957.50,3.891820192337036.50,3.9120230674743652.50,3.931825637817383.50,3.9512436389923096.50,3.97029185295105.50,3.9889841079711914.50,4.007333278656006.50
x,,,,,,,,,,,,,,,,,,,,,
y,NaN,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,1.609438,...,4.007333,4.007333,4.007333,4.007333,4.007333,4.007333,4.007333,4.007333,4.007333,4.007333
run,trial_nr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.529106,0.702815,0.804406,0.663587,0.544189,0.354976,0.273285,0.255804,0.288620,...,0.109866,0.110316,0.110721,0.111083,0.111409,0.111695,0.111944,0.112159,0.112341,0.112492
1,2,0.078678,0.131653,0.139261,0.137286,0.182603,0.171597,0.158266,0.162743,0.207638,...,0.331804,0.333673,0.336372,0.339818,0.343918,0.348631,0.353916,0.359764,0.366166,0.373103
1,3,0.019448,0.031065,0.042095,0.048884,0.066539,0.066745,0.066647,0.068167,0.070194,...,0.215350,0.208044,0.200728,0.193451,0.186227,0.179082,0.172038,0.165094,0.158266,0.151554


In [ ]:
level_x = pdf.columns.values[1:]
level_y = pdf.loc['y'].values[1:]

multi_index = pd.MultiIndex.from_arrays([level_x, level_y], names=['num_x', 'num_y'])

df = pd.DataFrame(, columns=multi_index)


In [9]:
pdf.columns.values[1:]

array(['1.6094379425048828', '1.7917594909667969', '1.945910096168518',
       ..., '3.97029185295105.50', '3.9889841079711914.50',
       '4.007333278656006.50'], dtype=object)

In [11]:
pdf.loc['y'].values[1:]

array([1.6094379425048828, 1.6094379425048828, 1.6094379425048828, ...,
       4.007333278656006, 4.007333278656006, 4.007333278656006],
      dtype=object)

In [30]:
pdf.reset_index().columns[2:]

Index(['1.6094379425048828', '1.7917594909667969', '1.945910096168518',
       '2.079441547393799', '2.1972246170043945', '2.3025851249694824',
       '2.397895336151123', '2.4849066734313965', '2.5649492740631104',
       '2.6390573978424072',
       ...
       '3.828641414642334.50', '3.8501474857330322.50',
       '3.8712010383605957.50', '3.891820192337036.50',
       '3.9120230674743652.50', '3.931825637817383.50',
       '3.9512436389923096.50', '3.97029185295105.50', '3.9889841079711914.50',
       '4.007333278656006.50'],
      dtype='object', length=2601)

In [21]:
pdf.stack('x')

KeyError: 'Requested level (x) does not match index name (None)'

In [ ]:
def get_decoding_info(subject, session=1,n_stim=1,  bids_folder='/data/ds-dnumrisk',key = 'decoded_pdfs.volume', mask='NPC_R', n_voxels='select'): # 

    subject = f'{subject:02d}'
    
    key = f'decoded_pdfs_stim{n_stim}.volume.cv_vselect.denoise'
    fn = f'sub-{subject}_ses-{session}_mask-{mask}_space-T1w_pars.tsv'

    pdf = op.join(bids_folder, 'derivatives', key, f'sub-{subject}', 'func', fn)

    if op.exists(pdf):
        pdf = pd.read_csv(pdf, sep='\t', index_col=[0])
        pdf.columns = pdf.columns.astype(float)
        pdf = pdf.loc[:, np.log(5):np.log(28*4)] # restrict range to actually presensted numeroisities

        E = (pdf*pdf.columns.values[np.newaxis, :] / pdf.sum(1).values[:, np.newaxis]).sum(1)

        E = pd.concat((E,), keys=[(int(subject), int(session), mask, n_voxels)],
        names=['subject', 'session', 'mask', 'n_voxels']).to_frame('E')

        pdf /= np.trapz(pdf, pdf.columns.astype(float))[:, np.newaxis] #normalizing

        E['sd'] = np.trapz(np.abs(E.values - pdf.columns.astype(float).values[np.newaxis, :]) * pdf, pdf.columns, axis=1)
        #print('succesfully predicted')

        return E
    else:
        print(pdf)
        return pd.DataFrame(np.zeros((0, 0)))